In [2]:
import pandas as pd
from tqdm import tqdm
import ast

In [3]:
df = pd.read_csv('../data/players_infos.csv')

In [4]:
tactics = pd.read_csv('../data/tactical_systems copy.csv')

In [5]:
team = 'Manchester City'
season = 202425

In [6]:
teams_df = df[['Season', 'TEAM_ID', 'Team', 'League']]
teams_df = teams_df.merge(tactics[['Coach', 'Most Used System', 'Season', 'Team_ID']], how='left', left_on=['Season', 'TEAM_ID'], right_on=['Season', 'Team_ID']).drop('Team_ID', axis=1)

In [7]:
teams_df_train = teams_df[(teams_df.Season == season) & (teams_df.Team == team)]
teams_df_train = (teams_df_train.drop_duplicates())

In [8]:
ly_tactics = []
for i, vals in tqdm(teams_df_train.iterrows()):
    last_year_info = teams_df_train[(teams_df_train.Season == vals['Season']) & (teams_df_train.TEAM_ID == vals['TEAM_ID'])]
    if len(last_year_info)>0:
        
        last_year_tactics = last_year_info['Most Used System'].tail(1).values[0]
    else:
        last_year_tactics = vals['Most Used System']
    ly_tactics.append(last_year_tactics)

1it [00:00, 498.25it/s]


In [9]:
teams_df_train['LAST_YEAR_TACTICS'] = ly_tactics

In [10]:
tactics_pos = pd.read_csv('../data/Target Score Chart - MPT.csv')

In [11]:
tactics_pos = tactics_pos.iloc[:, 1:]
tactics_pos.columns = [a.strip() for a in tactics_pos.iloc[0].values] 
tactics_pos = tactics_pos.iloc[1:]

In [12]:
tactics_pos['Formation'] = tactics_pos.Formation.str.split(' ').str[0]
tactics_df = pd.pivot_table(tactics_pos, index='Formation', columns='Position', values='Ideal Number')
cor_order = ['GK', 'CB', 'RB', 'LB', 'DM', 'CM', 'LM', 'RM', 'RW', 'LW', 'ST']
tactics_df = tactics_df[cor_order]

In [13]:
tactics_df = tactics_df.reset_index()

In [14]:
new_tactics = pd.DataFrame([['3-4-2-1', 3, 5, 2, 2, 2, 4, 0, 0, 2, 2, 2],
 ['4-1-4-1', 3, 4, 2, 2, 2, 4, 0, 0, 2, 2, 2]], columns=tactics_df.columns)

In [15]:
tactics_df = pd.concat([tactics_df, new_tactics], axis=0)

In [16]:
teams_df_train['LAST_YEAR_TACTICS'] = [str(a).split("'")[-2] if '[' in a else a for a in teams_df_train['LAST_YEAR_TACTICS'].values]

In [17]:
teams_df_train['LAST_YEAR_TACTICS'] = teams_df_train['LAST_YEAR_TACTICS'].str.split(' ').str[0]

In [18]:
tactics_df.columns = [a+'_ideal' if len(a) == 2 else a for a in tactics_df.columns.values]

In [19]:
teams_df_info = teams_df_train.merge(tactics_df, how='left', left_on = 'LAST_YEAR_TACTICS', right_on = 'Formation', suffixes=['', '_ideal'])

In [20]:
def find_cap(s):
    pos = ''.join([char for char in s if char.isupper()])
    if pos == 'G':
        pos = 'GK'
    return pos
df['POS_CODE'] = df.Position.apply(lambda x: find_cap(x))

In [21]:
df.POS_CODE = df.POS_CODE.str.replace('CF', 'ST').str.replace('SS', 'ST')
df.POS_CODE = df.POS_CODE.str.replace('AM', 'CM')

In [22]:
df = df[~df.POS_CODE.isin(['A', 'D', 'M'])]

In [23]:
res_df_full = pd.DataFrame()
for i, vals in tqdm(teams_df_info.iterrows()):
    sub_df = df[(df.TEAM_ID == vals['TEAM_ID']) & (df.Season ==  vals['Season'])]
    
    res_df = pd.pivot_table(sub_df.groupby('POS_CODE').size().to_frame('current_pos_count').reset_index(), columns = 'POS_CODE', values='current_pos_count')
    res_df.columns = [a+'_current' for a in res_df.columns]
    res_df['TEAM_ID'] = vals['TEAM_ID']
    res_df['Season'] = vals['Season']
    
    res_df_full = pd.concat([res_df_full, res_df])

1it [00:00, 142.89it/s]


In [24]:
res_df_full = res_df_full.fillna(0)

teams_df_info = teams_df_info.merge(res_df_full, how='left', on=['TEAM_ID', 'Season'])

In [25]:
teams_df_info

,Season,TEAM_ID,Team,League,Coach,Most Used System,LAST_YEAR_TACTICS,Formation,GK_ideal,CB_ideal,...,ST_ideal,CB_current,CM_current,DM_current,GK_current,LB_current,LW_current,RB_current,RW_current,ST_current
0,202425.0,281,Manchester City,GB1,Pep Guardiola,3-4-2-1,3-4-2-1,3-4-2-1,3.0,5.0,...,2.0,4,7,1,3,2,2,3,3,1


In [45]:
base_cols = ['Season', 'Team']
backup_df = teams_df_info[(teams_df_info.Team == team) & (teams_df_info.Season == season)]

ideal_cols = [a for a in backup_df if '_ideal' in a ]
current_cols = [a for a in backup_df if '_current' in a ]


In [49]:
final_df = pd.concat([backup_df[ideal_cols].T.reset_index(),backup_df[current_cols].T.reset_index()], axis=1)
final_df

,index,4418,index,4418
0,GK_ideal,3.0,CB_current,4.0
1,CB_ideal,5.0,CM_current,7.0
2,RB_ideal,2.0,DM_current,1.0
3,LB_ideal,2.0,GK_current,3.0
4,DM_ideal,2.0,LB_current,2.0
5,CM_ideal,4.0,LW_current,2.0
6,LM_ideal,0.0,RB_current,3.0
7,RM_ideal,0.0,RW_current,3.0
8,RW_ideal,2.0,ST_current,1.0
9,LW_ideal,2.0,RM_current,0.0


In [51]:
final_df.iloc[:,[1, 3]]

,4418,4418
0,3.0,4.0
1,5.0,7.0
2,2.0,1.0
3,2.0,3.0
4,2.0,2.0
5,4.0,2.0
6,0.0,3.0
7,0.0,3.0
8,2.0,1.0
9,2.0,0.0


In [56]:
df.head(30)

,Season,Team,#,Link,PLAYER_ID,Player Name,Position,Age,Appearences,Played Games,...,Yellow,Second Yellow,Red,Come From Bench,Substituted,PPG,Minute Played,TEAM_ID,League,POS_CODE
0,201718.0,Manchester City,31.0,https://www.transfermarkt.co.uk/ederson/leistu...,238223.0,Ederson,Goalkeeper,23.0,38.0,36.0,...,1.0,0.0,0.0,0.0,1.0,2.61,3.195,281,GB1,GK
1,201718.0,Manchester City,54.0,https://www.transfermarkt.co.uk/angusgunn/leis...,201574.0,Angus Gunn,Goalkeeper,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000,281,GB1,GK
2,201718.0,Manchester City,0.0,https://www.transfermarkt.co.uk/joehart/leistu...,40204.0,Joe Hart,Goalkeeper,30.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000,281,GB1,GK
3,201718.0,Manchester City,1.0,https://www.transfermarkt.co.uk/claudiobravo/l...,40423.0,Claudio Bravo,Goalkeeper,34.0,38.0,3.0,...,0.0,0.0,0.0,1.0,0.0,3.00,225.000,281,GB1,GK
4,201718.0,Manchester City,5.0,https://www.transfermarkt.co.uk/johnstones/lei...,186590.0,John Stones,CentreBack,23.0,22.0,18.0,...,0.0,0.0,0.0,2.0,5.0,2.61,1.306,281,GB1,CB
5,201718.0,Manchester City,35.0,https://www.transfermarkt.co.uk/oleksandrzinch...,203853.0,Oleksandr Zinchenko,LeftBack,20.0,20.0,8.0,...,1.0,0.0,0.0,2.0,3.0,3.00,533.000,281,GB1,LB
6,201718.0,Manchester City,0.0,https://www.transfermarkt.co.uk/tosinadarabioy...,258878.0,Tosin Adarabioyo,CentreBack,19.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.000,281,GB1,CB
7,201718.0,Manchester City,14.0,https://www.transfermarkt.co.uk/aymericlaporte...,176553.0,Aymeric Laporte,CentreBack,23.0,14.0,9.0,...,1.0,0.0,0.0,0.0,1.0,3.00,807.000,281,GB1,CB
8,201718.0,Manchester City,2.0,https://www.transfermarkt.co.uk/kylewalker/lei...,95424.0,Kyle Walker,RightBack,27.0,36.0,32.0,...,2.0,1.0,0.0,0.0,2.0,2.66,2.787,281,GB1,RB
9,201718.0,Manchester City,3.0,https://www.transfermarkt.co.uk/danilo/leistun...,145707.0,Danilo,CentreBack,25.0,36.0,23.0,...,2.0,0.0,0.0,10.0,2.0,2.48,1.341,281,GB1,CB
